In [ ]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 5.35 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 23.4 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-01-26 08:47:29--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c5:2ef4, 2406:da00:ff00::

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

hgtk : https://github.com/bluedisk/hangul-toolkit

In [ ]:
# 한글 자모 단위 처리 패키지 설치
!pip install hgtk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# fasttext 설치
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!pip install .

fatal: destination path 'fastText' already exists and is not an empty directory.
/content/fastText
make: Nothing to be done for 'opt'.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/fastText
  Preparing metadata (setup.py) ... done
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=4385087 sha256=34c43ba84244fb4dd7bb67914f4bd3f7e457f50a458905c8e6f8ceca51f29b57
  Stored in directory: /tmp/pip-ephem-wheel-cache-1a7ypjss/wheels/a4/2f/6a/91d479a807787e092f667baec5df08801b0558f5676427f5a9
Successfully built fasttext
  Attempting uninstall: fasttext
    Found existing installation: fasttext 0.9.2
    Uninstalling fasttext-0.9.2:
      Successfully uninstalled fasttext-0.9.2


# 1. 데이터 로드

In [ ]:
import re
import pandas as pd
import urllib.request
from tqdm import tqdm
import hgtk
from konlpy.tag import Mecab

In [ ]:
# 네이버 쇼핑 리뷰
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

('ratings_total.txt', <http.client.HTTPMessage at 0x7f9c8dfbc430>)

In [ ]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data)) # 전체 리뷰 개수 출력

전체 리뷰 개수 : 200000


In [ ]:
total_data[:5]

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


# 2. hgtk 튜토리얼

In [ ]:
# 한글인지 체크
hgtk.checker.is_hangul('ㄱ')

True

In [ ]:
# 한글인지 체크
hgtk.checker.is_hangul('28')

False

In [ ]:
# 음절을 초성, 중성, 종성으로 분해
hgtk.letter.decompose('남')

('ㄴ', 'ㅏ', 'ㅁ')

In [ ]:
# 초성, 중성을 결합
hgtk.letter.compose('ㄴ', 'ㅏ')

'나'

In [ ]:
# 초성, 중성, 종성을 결합
hgtk.letter.compose('ㄴ', 'ㅏ', 'ㅁ')

'남'

In [ ]:
# 한글이 아닌 입력에 대해서는 에러 발생.
hgtk.letter.decompose('1')

NotHangulException: ignored

In [ ]:
# 결합할 수 없는 상황에서는 에러 발생
hgtk.letter.compose('ㄴ', 'ㅁ', 'ㅁ')

# 3. 데이터 전처리

In [ ]:
def word_to_jamo(token):
  def to_special_token(jamo):
    if not jamo:
      return '-'
    else:
      return jamo

  decomposed_token = ''
  for char in token:
    try:
      # char(음절)을 초성, 중성, 종성으로 분리
      cho, jung, jong = hgtk.letter.decompose(char)

      # 자모가 빈 문자일 경우 특수문자 -로 대체
      cho = to_special_token(cho)
      jung = to_special_token(jung)
      jong = to_special_token(jong)
      decomposed_token = decomposed_token + cho + jung + jong

    # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
    except Exception as exception:
      if type(exception).__name__ == 'NotHangulException':
        decomposed_token += char
    
  # 단어 토큰의 자모 단위 분리 결과를 추가
  return decomposed_token

In [ ]:
word_to_jamo('남동생')

'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

In [ ]:
word_to_jamo('여동생')

'ㅇㅕ-ㄷㅗㅇㅅㅐㅇ'

In [ ]:
mecab = Mecab()

In [ ]:
print(mecab.morphs('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['선물', '용', '으로', '빨리', '받', '아서', '전달', '했어야', '하', '는', '상품', '이', '었', '는데', '머그', '컵', '만', '와서', '당황', '했', '습니다', '.']


In [ ]:
def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]

In [ ]:
print(tokenize_by_jamo('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['ㅅㅓㄴㅁㅜㄹ', 'ㅇㅛㅇ', 'ㅇㅡ-ㄹㅗ-', 'ㅃㅏㄹㄹㅣ-', 'ㅂㅏㄷ', 'ㅇㅏ-ㅅㅓ-', 'ㅈㅓㄴㄷㅏㄹ', 'ㅎㅐㅆㅇㅓ-ㅇㅑ-', 'ㅎㅏ-', 'ㄴㅡㄴ', 'ㅅㅏㅇㅍㅜㅁ', 'ㅇㅣ-', 'ㅇㅓㅆ', 'ㄴㅡㄴㄷㅔ-', 'ㅁㅓ-ㄱㅡ-', 'ㅋㅓㅂ', 'ㅁㅏㄴ', 'ㅇㅘ-ㅅㅓ-', 'ㄷㅏㅇㅎㅘㅇ', 'ㅎㅐㅆ', 'ㅅㅡㅂㄴㅣ-ㄷㅏ-', '.']


In [ ]:
tokenized_data = []

for sample in tqdm(total_data['reviews'].to_list()):
    tokenzied_sample = tokenize_by_jamo(sample) # 자소 단위 토큰화
    tokenized_data.append(tokenzied_sample)

100%|██████████| 200000/200000 [01:17<00:00, 2568.64it/s]


In [ ]:
len(tokenized_data)

200000

In [ ]:
tokenized_data[0]

['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-', 'ㄱㅗ-', 'ㄱㅜㅅ']

In [ ]:
def jamo_to_word(jamo_sequence):
  tokenized_jamo = []
  index = 0
  
  # 1. 초기 입력
  # jamo_sequence = 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

  while index < len(jamo_sequence):
    # 문자가 한글(정상적인 자모)이 아닐 경우
    if not hgtk.checker.is_hangul(jamo_sequence[index]):
      tokenized_jamo.append(jamo_sequence[index])
      index = index + 1

    # 문자가 정상적인 자모라면 초성, 중성, 종성을 하나의 토큰으로 간주.
    else:
      tokenized_jamo.append(jamo_sequence[index:index + 3])
      index = index + 3

  # 2. 자모 단위 토큰화 완료
  # tokenized_jamo : ['ㄴㅏㅁ', 'ㄷㅗㅇ', 'ㅅㅐㅇ']
  
  word = ''
  try:
    for jamo in tokenized_jamo:

      # 초성, 중성, 종성의 묶음으로 추정되는 경우
      if len(jamo) == 3:
        if jamo[2] == "-":
          # 종성이 존재하지 않는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1])
        else:
          # 종성이 존재하는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
      # 한글이 아닌 경우
      else:
        word = word + jamo

  # 복원 중(hgtk.letter.compose) 에러 발생 시 초기 입력 리턴.
  # 복원이 불가능한 경우 예시) 'ㄴ!ㅁㄷㅗㅇㅅㅐㅇ'
  except Exception as exception:  
    if type(exception).__name__ == 'NotHangulException':
      return jamo_sequence

  # 3. 단어로 복원 완료
  # word : '남동생'

  return word

In [ ]:
jamo_to_word('ㄴㅏㅁㄷㅗㅇㅅㅐㅇ')

'남동생'

# 4. FastText

In [ ]:
import fasttext

In [ ]:
with open('tokenized_data.txt', 'w') as out:
  for line in tqdm(tokenized_data, unit=' line'):
    out.write(' '.join(line) + '\n')

100%|██████████| 200000/200000 [00:00<00:00, 363404.67 line/s]


In [ ]:
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')

In [ ]:
model.save_model("fasttext.bin")

In [ ]:
model = fasttext.load_model("fasttext.bin")

In [ ]:
model[word_to_jamo('남동생')] # 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

array([-0.22610371, -0.30660358, -0.32662258,  0.72500163,  0.12612171,
       -1.2639583 , -0.18436524, -0.5719366 , -1.0128864 , -0.06393725,
        0.152386  , -0.38007712, -0.72380483, -0.09851937, -0.45947054,
        0.04964648,  0.64551294, -0.09582338, -0.03914408, -0.4805486 ,
        0.925344  , -0.2491744 , -0.45504475, -0.02600918, -0.15205085,
        0.48291105, -0.37986383,  0.34643406,  0.59756   ,  0.4526392 ,
        0.19838001,  1.2457352 ,  0.295814  , -1.26712   ,  0.21076524,
        0.7044634 , -0.10213161, -0.71973974, -0.04362598,  0.15851249,
       -0.39800492,  0.9522386 ,  0.1978278 ,  0.8664345 , -0.27567104,
       -0.5154622 ,  0.07898171, -0.0304397 ,  0.6073195 ,  1.1716179 ,
        0.2819162 ,  1.0266525 , -0.8328321 , -0.3399172 ,  0.2376753 ,
        0.1589474 ,  0.01766292, -0.67157215,  0.19347401, -0.7422794 ,
       -0.17230903, -0.16539897,  0.96320105,  0.4920474 ,  0.56401324,
        0.67648095,  0.09782839,  0.2591442 ,  0.1110479 ,  0.34

In [ ]:
model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)

[(0.8911885619163513, 'ㄷㅗㅇㅅㅐㅇ'),
 (0.8704726099967957, 'ㄴㅏㅁㅊㅣㄴ'),
 (0.7694569826126099, 'ㄴㅏㅁㅍㅕㄴ'),
 (0.7426445484161377, 'ㅅㅐㅇㅇㅣㄹ'),
 (0.7372080683708191, 'ㄴㅏㅁㅇㅏ-'),
 (0.7304610013961792, 'ㅊㅣㄴㄱㅜ-'),
 (0.7281805276870728, 'ㄴㅏㄴㅅㅐㅇ'),
 (0.7148119807243347, 'ㅈㅜㅇㅎㅏㄱㅅㅐㅇ'),
 (0.7100965976715088, 'ㅈㅗ-ㅋㅏ-'),
 (0.7070216536521912, 'ㅇㅓㄴㄴㅣ-')]

In [ ]:
def transform(word_sequence):
  return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))

[('동생', 0.8911885619163513), ('남친', 0.8704726099967957), ('남편', 0.7694569826126099), ('생일', 0.7426445484161377), ('남아', 0.7372080683708191), ('친구', 0.7304610013961792), ('난생', 0.7281805276870728), ('중학생', 0.7148119807243347), ('조카', 0.7100965976715088), ('언니', 0.7070216536521912)]


# 5. Word2Vec

In [ ]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']


In [ ]:
tokenized_data2 = []
for sentence in tqdm(total_data['reviews'].to_list()):
    tokenized_sentence = mecab.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data2.append(stopwords_removed_sentence)

100%|██████████| 200000/200000 [00:31<00:00, 6402.88it/s]


In [ ]:
from gensim.models import Word2Vec

model2 = Word2Vec(sentences = tokenized_data2, size = 1000, window = 5, min_count = 5, workers = 4, sg = 0)

In [ ]:
# 완성된 임베딩 매트릭스의 크기 확인
model2.wv.vectors.shape

(14959, 1000)

# 6. FastText Word2Vec 결과 비교

In [ ]:
## FastText
print(transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))

[('동생', 0.8911885619163513), ('남친', 0.8704726099967957), ('남편', 0.7694569826126099), ('생일', 0.7426445484161377), ('남아', 0.7372080683708191), ('친구', 0.7304610013961792), ('난생', 0.7281805276870728), ('중학생', 0.7148119807243347), ('조카', 0.7100965976715088), ('언니', 0.7070216536521912)]


In [ ]:
## Word2Vec
print(model2.wv.most_similar("남동생"))

[('양가', 0.7742542028427124), ('친한', 0.7253313064575195), ('엄니', 0.7234327793121338), ('개업', 0.7228173017501831), ('남친', 0.7066282033920288), ('딸래미', 0.7036538124084473), ('하시', 0.7030436992645264), ('어머님께', 0.6994115710258484), ('연말', 0.6991868615150452), ('셔셔', 0.6928422451019287)]
